## MLJ Basics

This notebook shows the basics tools needed to deal with a classification task in MLJ.

In particular it shows:

- How to load a model
- How to fit a model
- How to predict using a fitted model
- How to evaluate a model


In [2]:
using RDatasets, MLJ, NearestNeighbors, MLJModels
iris = dataset("datasets", "iris")
first(iris, 3) |> pretty

┌─────────────┬────────────┬─────────────┬────────────┬────────────────────────────────┐
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │ Species                        │
│ Float64     │ Float64    │ Float64     │ Float64    │ CategoricalValue{String,UInt8} │
│ Continuous  │ Continuous │ Continuous  │ Continuous │ Multiclass{3}                  │
├─────────────┼────────────┼─────────────┼────────────┼────────────────────────────────┤
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │ setosa                         │
│ 4.9         │ 3.0        │ 1.4         │ 0.2        │ setosa                         │
│ 4.7         │ 3.2        │ 1.3         │ 0.2        │ setosa                         │
└─────────────┴────────────┴─────────────┴────────────┴────────────────────────────────┘


In [6]:
using Pkg
Pkg.status("MLJ")

Status `~/.julia/environments/v1.5/Project.toml`
  [add582a8] MLJ v0.15.0


In [7]:
y, X = unpack(iris, ==(:Species), colname -> true)
first(X, 1) |> pretty

┌─────────────┬────────────┬─────────────┬────────────┐
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │
│ Float64     │ Float64    │ Float64     │ Float64    │
│ Continuous  │ Continuous │ Continuous  │ Continuous │
├─────────────┼────────────┼─────────────┼────────────┤
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │
└─────────────┴────────────┴─────────────┴────────────┘


In [8]:
println(typeof(X))
println(typeof(y))

DataFrame
CategoricalArrays.CategoricalArray{String,1,UInt8,String,CategoricalArrays.CategoricalValue{String,UInt8},Union{}}


In [9]:
for m in models(matching(X, y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 30), "($(m.package_name))")
    end
end

AdaBoostClassifier            (ScikitLearn)
AdaBoostStumpClassifier       (DecisionTree)
BaggingClassifier             (ScikitLearn)
BayesianLDA                   (MultivariateStats)
BayesianLDA                   (ScikitLearn)
BayesianQDA                   (ScikitLearn)
BayesianSubspaceLDA           (MultivariateStats)
ConstantClassifier            (MLJModels)
DecisionTreeClassifier        (DecisionTree)
DummyClassifier               (ScikitLearn)
EvoTreeClassifier             (EvoTrees)
ExtraTreesClassifier          (ScikitLearn)
GaussianNBClassifier          (NaiveBayes)
GaussianNBClassifier          (ScikitLearn)
GaussianProcessClassifier     (ScikitLearn)
GradientBoostingClassifier    (ScikitLearn)
KNNClassifier                 (NearestNeighbors)
KNeighborsClassifier          (ScikitLearn)
LDA                           (MultivariateStats)
LGBMClassifier                (LightGBM)
LogisticCVClassifier          (ScikitLearn)
LogisticClassifier            (MLJLinearModels)
LogisticClas

## Training a KNN

### Choosing a model

In [10]:
knn = @load KNNClassifier verbosity = 0

KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @028

We can define specific atributes of the model setting them direcly to the loaded object.

For example, we can set the `leafsize` to be 5:

In [11]:
knn.leafsize = 10

10

In [12]:
knn

KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @028

### Defining a machine with a model and the data

Now we will define a `MLJ.Machine` object that will contain 3 objects:

- The model `knn`
- The input data `X`
- The output data `y`

In [79]:
println(typeof(X))
println(typeof(y))

DataFrame
CategoricalArrays.CategoricalArray{String,1,UInt8,String,CategoricalArrays.CategoricalValue{String,UInt8},Union{}}


Let us define the machine with the function `machine`

In [80]:
m_knn = machine(knn, X, y)

Machine{KNNClassifier} @595 trained 0 times.
  args: 
    1:	Source @536 ⏎ `Table{AbstractArray{Continuous,1}}`
    2:	Source @465 ⏎ `AbstractArray{Multiclass{3},1}`


Now we have a `Machine` object containing a `KNNClassifier`

In [81]:
typeof(m_knn)

Machine{MLJModels.NearestNeighbors_.KNNClassifier}

Since the machine already has all the training data we can specify a set of indices for training and one set for test.

A very handy function to generate train and test splits is **`partition`** which can take as input 

- `UnitRange`  (ex: `1:10`) 
- `Array` (ex:`[1,2,3,4,5,6,7,8,9,10]`) 
- `AbstractUnitRange` (ex: `eachindex(y)`)

To generate a train/test split. It returns two arrays with the expected partitions.

If you want a reproducible partition you can use Random.seed!(some_integer) so that the partition will be allways the same
 
 
#### Julia example
```
train_test =    partition(1:10, 0.8, shuffle=true)
train_test 

([4, 3, 9, 7, 2, 1, 6, 8], [10, 5])
```

#### Sklearn equivalent


```
train_test =  sklearn.model_selection.train_test_split(range(10), 
                                                       train_size=0.8, 
                                                       shuffle=True, 
                                                       random_state=123)
train_test
[[7, 5, 8, 3, 1, 6, 9, 2], [4, 0]]
``` 


The following version to generate a train, test split are equivalent

In [82]:
using Random
Random.seed!(123)
train_ind, test_ind = partition(Array(1:length(y)), 0.7, shuffle=true)

([125, 100, 130, 9, 70, 148, 39, 64, 6, 107  …  134, 114, 52, 74, 44, 61, 83, 18, 122, 26], [97, 78, 30, 108, 101, 24, 85, 91, 135, 96  …  112, 144, 140, 72, 109, 41, 106, 147, 47, 5])

In [83]:
using Random
Random.seed!(123)
train_ind, test_ind = partition(eachindex(y), 0.7, shuffle=true)

([125, 100, 130, 9, 70, 148, 39, 64, 6, 107  …  134, 114, 52, 74, 44, 61, 83, 18, 122, 26], [97, 78, 30, 108, 101, 24, 85, 91, 135, 96  …  112, 144, 140, 72, 109, 41, 106, 147, 47, 5])

In [84]:
using Random
Random.seed!(123)
train_ind, test_ind = partition(1:length(y), 0.7, shuffle=true)

([125, 100, 130, 9, 70, 148, 39, 64, 6, 107  …  134, 114, 52, 74, 44, 61, 83, 18, 122, 26], [97, 78, 30, 108, 101, 24, 85, 91, 135, 96  …  112, 144, 140, 72, 109, 41, 106, 147, 47, 5])

## Training and predicting with a machine
We can train a machine using **`fit!`**  and we can specify the rows used with **`rows=training_ind`**

In [85]:
fit!(m_knn, rows=train_ind)

┌ Info: Training Machine{KNNClassifier} @595.
└ @ MLJBase /Users/davidbuchaca1/.julia/packages/MLJBase/5TNcr/src/machines.jl:319


Machine{KNNClassifier} @595 trained 1 time.
  args: 
    1:	Source @536 ⏎ `Table{AbstractArray{Continuous,1}}`
    2:	Source @465 ⏎ `AbstractArray{Multiclass{3},1}`


**`fitted_params(machine)`** allows us to view the fitted parameters of a `machine`.

In [86]:
fitted_params(m_knn) |> print

(tree = KDTree{StaticArrays.SArray{Tuple{4},Float64,1,4},Euclidean,Float64}
  Number of points: 105
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true,)

**`predict`** allows us to get  predictions from a machine. 

There are two different approaches:

- **`predict(machine, rows=test_ind)`**: Use the internal dataset that provided to the machine specifying the rows from which to make predictions.


- **`predict(machine, X_df)`**: Use a dataframe `X_df` from which to make predictions.



In [87]:
ŷ_test  = predict(m_knn, rows=test_ind);
ŷ_train = predict(m_knn, rows=train_ind);

In [93]:
ŷ_train = predict(m_knn, X[train_ind,:])
ŷ_test  = predict(m_knn, X[test_ind,:]);

In [104]:
y_train = y[train_ind]
y_test = y[test_ind];

In [105]:
typeof(ŷ_train)

MLJBase.UnivariateFiniteArray{Multiclass{3},String,UInt8,Float64,1}

In [106]:
ŷ_train[3]

UnivariateFinite{Multiclass{3}}(setosa=>0.0, versicolor=>0.0, virginica=>1.0)

It is important to note that `UnivariateFinite` cannot be directly compared with a `CategoricalArray`. 

In [107]:
ŷ_train[1]

UnivariateFinite{Multiclass{3}}(setosa=>0.0, versicolor=>0.0, virginica=>1.0)

In [108]:
y_train[]

LoadError: BoundsError: attempt to access 105-element CategoricalArrays.CategoricalArray{String,1,UInt8} at index []

In [109]:
ŷ_train[1] == y_train[1]

false

### Predictions for classification algorithms in  MLJ

Notice that in MLJ machine predictions for classification problems return an Array of the following form: `MLJBase.UnivariateFiniteArray{Multiclass{3},String,UInt8,Float64,1}`.
This type might be a bit surprising, let's see the details.

You can interpret `ŷ_test[k]` as an array of length `n_classes`. But users can't do `ŷ_test[k][c]` directly.

A component `c` in this "array" contains the probability of example `k` beeing from the class indexed by integer `c`. 

For example, the following line tells us that the probability of `versicolor` is 0.6 and the probability of `virginica` is 0.4.

```julia
ŷ_test[2]
UnivariateFinite{Multiclass{3}}(setosa=>0.0, versicolor=>0.6, virginica=>0.4)
```

Note that the output of predict is a **`MLJBase.UnivariateFiniteArray`** and each element is of type **`UnivariateFinite`** which contains the predicted probabilities for each class given the input of the predict.


There are several advantadges for using `UnivariateFinite`  instead of a vector of floats **TODO:EXPAND THIS**.


##### Sklearn equivalent

In sklearn, `model.predict(X)` returns a `np.array` containing the predicted class labels (allawys coded as integers from 0 to `n_classes-1` ).
 
Notice that `predict(tree_machine, X[test_ind,:])` in MLJ would be pretty much the same as `tree_machine.predict_proba(X[test,:])` in Sklearn. The main difference is that Sklearn would return a numpy array of float values whereas MLJ returns a "weird" array of `UnivariateFinite{Multiclass{3}}` values.




In [110]:
ŷ_test[2]

UnivariateFinite{Multiclass{3}}(setosa=>0.0, versicolor=>0.6, virginica=>0.4)

In [111]:
ŷ_test[2].prob_given_ref

OrderedCollections.LittleDict{UInt8,Float64,Array{UInt8,1},Array{Float64,1}} with 3 entries:
  0x01 => 0.0
  0x02 => 0.6
  0x03 => 0.4

**`prob_given_ref.vals`** returns a vector with the probabilities.

In [112]:
ŷ_test[2].prob_given_ref.vals

3-element Array{Float64,1}:
 0.0
 0.6000000000000001
 0.4

**`mode(ŷ_test[k])`**  can be used to get the most likely class for example `k`

In [113]:
mode(ŷ_test[2])

CategoricalArrays.CategoricalValue{String,UInt8} "versicolor"

**`predict_mode(machine, X)`** returns a categorical array with the predicted classes from `X`

In [114]:
 predict_mode(m_knn, X[test_ind[1:3],:])

3-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "versicolor"
 "versicolor"
 "setosa"

we can make comparissons to check if the predicted class equals the true class

In [115]:
mode(ŷ_test[2]) == y_test[2]

true

Users need to be carefull not to confuse the previous line with the following one:

In [116]:
ŷ_test[2] == y_test[2]

false


In order to compute the accuracy between `ŷ_test` and `y_test` we could do

In [117]:
function accuracy(y,ŷ)
    accuracy = 0.
    for m in 1:length(y)
        accuracy += y[m] == mode(ŷ[m])
    end
    return accuracy/length(y)
end

accuracy(y_test,ŷ_test)

0.9777777777777777

We can also compute the accuracy as follows:

In [198]:
1 - misclassification_rate(mode.(ŷ_test),y_test)

0.9777777777777777

Note that there are other metrics that can be used

In [201]:
mean(cross_entropy(ŷ_test,y_test))

0.08064497513068349

In [202]:
mce = mean(cross_entropy(ŷ_test, y_test))
round(mce, digits=4)

0.0806

**`levels`** allow us to retrieve the "levels" or categories assigned to each possible value in a CategoricalArray

In [136]:
y_levels = levels(y)

3-element Array{String,1}:
 "setosa"
 "versicolor"
 "virginica"

**`pdf`**: Evaluates the probability density (mass).

```
pdf(d::MultivariateDistribution, x::AbstractArray)
```

Return the probability density of distribution d evaluated at x.

- If `x` is a vector, it returns the result as a scalar.
- If `x` is a matrix with n columns, it returns a vector `r` of length `n`, 
   where `r[i]` corresponds to `x[:,i]` (i.e. treating each column as a sample).


`pdf!(r, d, x)` will write the results to a pre-allocated array `r`.


##### Sklearn equivalent

In sklearn the ouptut array containing the probabilities of each class given the input are extracted using `model.predict_proba`

```
y_test_probs = model.predict_proba(X)
```


In [143]:
y_test_probs = pdf(ŷ_test[1:3], y_levels[1:3])

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.0  0.6  0.4
 1.0  0.0  0.0

## Tunning Hyperparameters

In [163]:
knn = @load KNNClassifier verbosity = 0

KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @998

In [164]:
knn

KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @998

In [165]:
K_range = range(knn, :K, lower=5, upper=20);

In [166]:
K_range

MLJBase.NumericRange(Int64, :K, ... )

Incidentally, a grid is generated internally "over the range" by calling the iterator method with an appropriate resolution:

In [167]:
iterator(K_range, 3)

3-element Array{Int64,1}:
  5
 12
 20

Now let us define a tunned model

In [240]:
self_tuning_knn = TunedModel(model=knn,
                             resampling = CV(nfolds=5),
                             tuning = Grid(resolution=5),
                             range = K_range);

┌ Info: No measure specified. Setting measure=LogLoss{Float64} @278. 
└ @ MLJTuning /Users/davidbuchaca1/.julia/packages/MLJTuning/6MZ7C/src/tuned_models.jl:222


In [241]:
m_self_tuning_knn = machine(self_tuning_knn, X, y)

Machine{ProbabilisticTunedModel{Grid,…}} @199 trained 0 times.
  args: 
    1:	Source @189 ⏎ `Table{AbstractArray{Continuous,1}}`
    2:	Source @977 ⏎ `AbstractArray{Multiclass{3},1}`


In [242]:
fit!(m_self_tuning_knn, rows=train_ind, verbosity=0)

Machine{ProbabilisticTunedModel{Grid,…}} @199 trained 1 time.
  args: 
    1:	Source @189 ⏎ `Table{AbstractArray{Continuous,1}}`
    2:	Source @977 ⏎ `AbstractArray{Multiclass{3},1}`


In [243]:
fitted_params(m_self_tuning_knn)

(best_model = KNNClassifier @608,
 best_fitted_params = (tree = KDTree{StaticArrays.SArray{Tuple{4},Float64,1,4},Euclidean,Float64}
  Number of points: 105
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true,),)

In [223]:
fitted_params(m_self_tuning_knn).best_model

KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @332

**`report`** allows us to access a thorough report of the tuning process

In [224]:
report(m_self_tuning_knn)

(best_model = KNNClassifier @332,
 best_history_entry = (model = KNNClassifier @332,
                       measure = LogLoss{Float64}[LogLoss{Float64} @278],
                       measurement = [0.09329713262922215],
                       per_fold = [[0.15522370568516472, 0.06795792169714998, 0.06682856328680156, 0.06488465878583707, 0.11159081369115739]],),
 history = NamedTuple{(:model, :measure, :measurement, :per_fold),Tuple{MLJModels.NearestNeighbors_.KNNClassifier,Array{LogLoss{Float64},1},Array{Float64,1},Array{Array{Float64,1},1}}}[(model = KNNClassifier @017, measure = [LogLoss{Float64} @278], measurement = [0.10839692609461807], per_fold = [[0.12145588972882732, 0.05619179429302963, 0.15541404641084935, 0.05619179429302963, 0.15273110574735446]]), (model = KNNClassifier @597, measure = [LogLoss{Float64} @278], measurement = [0.1670047290499682], per_fold = [[0.18149729477405077, 0.10005618653945388, 0.21909874840716415, 0.14570504613580174, 0.18866636939337042]]), (model =

**`report(machine).history`** contains the per fold meatures stored during training

In [225]:
report(m_self_tuning_knn).history

5-element Array{NamedTuple{(:model, :measure, :measurement, :per_fold),Tuple{MLJModels.NearestNeighbors_.KNNClassifier,Array{LogLoss{Float64},1},Array{Float64,1},Array{Array{Float64,1},1}}},1}:
 (model = KNNClassifier @017, measure = [LogLoss{Float64} @278], measurement = [0.10839692609461807], per_fold = [[0.12145588972882732, 0.05619179429302963, 0.15541404641084935, 0.05619179429302963, 0.15273110574735446]])
 (model = KNNClassifier @597, measure = [LogLoss{Float64} @278], measurement = [0.1670047290499682], per_fold = [[0.18149729477405077, 0.10005618653945388, 0.21909874840716415, 0.14570504613580174, 0.18866636939337042]])
 (model = KNNClassifier @384, measure = [LogLoss{Float64} @278], measurement = [0.11642670571866609], per_fold = [[0.13751290362838622, 0.06696030644151882, 0.16084031807589744, 0.06367493160880684, 0.1531450688387212]])
 (model = KNNClassifier @332, measure = [LogLoss{Float64} @278], measurement = [0.09329713262922215], per_fold = [[0.15522370568516472, 0.0679

Note that the measure used to evaluate a particular hyperparameter is the mean over the different folds

In [226]:
report(m_self_tuning_knn).history[1].measurement

1-element Array{Float64,1}:
 0.10839692609461807

This value can be computed diretcly from the `.per_fold` array

In [51]:
mean(report(m_self_tuning_knn).history[1].per_fold[1])

0.13665915288523797

In [57]:
#accuracy(y_test, predict(m_self_tuning_knn, X[test_ind,:]))

### Defining a custom metric for the hyperparameter selection process

In [235]:
#measures()

In [244]:
self_tuning_knn = TunedModel(model=knn,
                             resampling = CV(nfolds=5),
                             tuning = Grid(resolution=5),
                             range = K_range,
    measure=misclassification_rate,
    check_measure=false);

In [246]:
misclassification_rate

MisclassificationRate() @021